Google Merch Store - Analysis

This exercise contains example data from the Google Merchandise Store via the Google Analytics Demo Account, as well as some fabricated cost price data for practice purposes.

## Overview
### Datasets
- Transaction data from 2017-04 to 2017-06 available in multiple files split by month (transactions_*.csv)
    - Each purchase on the Google Merchandise Store is logged as a transaction with a unique Transaction ID.
    - Each transaction can be comprised of one or more Product SKUs, with varying quantities.
    - Avg. Price refers to the price of each product unit that was sold within that transaction.
    
    
- Cost price data from 2017-04 to 2017-06 available in a single file (cost_price_2017-04_2017-06.csv)
    - The accounting department provides a monthly record of the average cost price of a product unit per SKU.

### Goals
- Generate Top 20 Product Categories with the largest profit margin
- Generate Top 20 SKUs with the largest profit margin
### Actions Needed
- Datatype conversion whenever appropriate
- Rename original columns to a standardized format
    - **Date** => date
    - **Transaction ID** => transaction_id
    - **Product SKU** => sku
    - **Product** => product_name
    - **Product Category (Enhanced Ecommerce)** => product_category
    - **Quantity** => quantity
    - **Avg. Price** => price


### Importing and configuring libraries

In [26]:
import os
from datetime import datetime
import pandas as pd


# set pandas options to display all columns, or else columns would be displayed on multiple lines
pd.set_option('display.expand_frame_repr', False)


### Setting up paths and folders

In [27]:
# identify the current directory
CURRENT_DIR = os.getcwd()

# define the path of the directory where data would be read from, which is in {CURRENT_DIR}/data
READ_DIR = os.path.join(CURRENT_DIR, 'data')

# define the path of the directory where any potential files would be written to, which is in {CURRENT_DIR}/output
WRITE_DIR = os.path.join(CURRENT_DIR, 'output')

# checks if WRITE_DIR exists. create if it doesn't already exist
if not os.path.exists(WRITE_DIR):
    os.mkdir(WRITE_DIR)

### Define helper functions

In [28]:
# convert text string ("$x,xxx.xx") to float
def currency_to_float(input_value):
    return float(input_value.lstrip('$').replace(',', ''))

# convert datetime object to "<year>-<month>"
def datetime_to_year_month(input_value):
    return input_value.strftime('%Y-%m')

def date_parser(date_string):
    return datetime.strptime(date_string, '%Y%m%d')

### Define and read files

Data is split into multiple files. The analysis should be done across all data in the separate files.

In [29]:
# define variable to store the collated Dataframe
transactions_df = None

# list files in folder
file_list = os.listdir(READ_DIR)

for file_name in file_list:
    if file_name.startswith('transactions_') and file_name.endswith('.csv'):
        read_path = os.path.join(READ_DIR, file_name)

        df = pd.read_csv(
            read_path,
            parse_dates=['Date'],
            date_parser=date_parser,
            thousands=','
        )

        if transactions_df is None:
            transactions_df = df
        else:
            transactions_df = transactions_df.append(df, ignore_index=True)

print(df)

           Date  Transaction ID     Product SKU                                            Product Product Category (Enhanced Ecommerce)  Quantity Avg. Price
0    2017-04-05           24860  GGOEGGOA017399                                           Maze Pen                                Office      1000      $0.99
1    2017-04-05           24860  GGOEYFKQ020699                              YouTube Custom Decals                                Office      1000      $1.99
2    2017-04-05           24863  GGOEGHGH019699                                  Google Sunglasses                             Lifestyle       750      $3.50
3    2017-04-05           24863  GGOEGOCB017499                                Leatherette Journal                                Office       600     $10.99
4    2017-04-05           24860  GGOEGHPJ080110                                 Google 5-Panel Cap                              Headgear       500     $18.99
...         ...             ...             ...     

### Rename Columns

- **Date** => date
- **Transaction ID** => transaction_id
- **Product SKU** => sku
- **Product** => product_name
- **Product Category (Enhanced Ecommerce)** => product_category
- **Quantity** => quantity
- **Avg. Price** => price

In [30]:
renaming_map = {
    'Date': 'date',
    'Transaction ID': 'transaction_id',
    'Product SKU': 'sku',
    'Product': 'product_name',
    'Product Category (Enhanced Ecommerce)': 'product_category',
    'Quantity': 'quantity',
    'Avg. Price': 'price'
}

transactions_df = transactions_df.rename(columns=renaming_map)
print(transactions_df)

            date  transaction_id             sku                                       product_name product_category  quantity    price
0     2017-06-21           30876  GGOEGOAQ012899                            Ballpoint LED Light Pen           Office      1000    $2.50
1     2017-06-05           29657  GGOEGGOA017399                                           Maze Pen           Office       825    $0.79
2     2017-06-08           29898  GGOEGGOA017399                                           Maze Pen           Office       825    $0.79
3     2017-06-12           30174  GGOEGOAC021799                                 Ballpoint Pen Blue           Office       500    $0.99
4     2017-06-14           30377  GGOEGFKA022299                               Keyboard DOT Sticker           Office       500    $1.50
...          ...             ...             ...                                                ...              ...       ...      ...
13872 2017-04-30           26837  GGOEGHPA002910

### Modify data
- Convert **price** to float
- Add new column **year_month** derived from **date**

In [31]:
# convert price string to float
transactions_df['price'] = transactions_df['price'].apply(currency_to_float)

# add new column with year_month
transactions_df['year_month'] = transactions_df['date'].apply(datetime_to_year_month)

### Import cost data

In [32]:
cost_path = os.path.join(READ_DIR, 'cost_price_2017-04_2017-06.csv')
cost_df = pd.read_csv(cost_path)
print(cost_df)

     year_month             sku  cost_price
0       2017-04  GGOEAAAB034813        6.40
1       2017-04  GGOEAAAB034814        6.80
2       2017-04  GGOEAAAB034815        7.54
3       2017-04  GGOEAAAB034816        6.40
4       2017-04  GGOEAAAB034817       13.81
...         ...             ...         ...
2617    2017-06  GGOEYOCR077399        9.10
2618    2017-06  GGOEYOCR077799        5.04
2619    2017-06  GGOEYOCR078099        2.48
2620    2017-06  GGOEYOLR018699        3.79
2621    2017-06  GGOEYOLR080599        2.00

[2622 rows x 3 columns]


### Match cost price data for each sku for specific months

In [33]:
transactions_df = pd.merge(transactions_df, cost_df, how='left', on=['year_month', 'sku'])
print(transactions_df)

            date  transaction_id             sku                                       product_name product_category  quantity   price year_month  cost_price
0     2017-06-21           30876  GGOEGOAQ012899                            Ballpoint LED Light Pen           Office      1000    2.50    2017-06        0.88
1     2017-06-05           29657  GGOEGGOA017399                                           Maze Pen           Office       825    0.79    2017-06        0.37
2     2017-06-08           29898  GGOEGGOA017399                                           Maze Pen           Office       825    0.79    2017-06        0.37
3     2017-06-12           30174  GGOEGOAC021799                                 Ballpoint Pen Blue           Office       500    0.99    2017-06        0.64
4     2017-06-14           30377  GGOEGFKA022299                               Keyboard DOT Sticker           Office       500    1.50    2017-06        0.54
...          ...             ...             ...    

## Get Top 20 Product Categories with the highest profit margin

# Group then calculate profit margin

In [34]:
# group data by product_category
category_profit_margin_df = transactions_df.groupby(['product_category'], as_index=False).agg(
    {
        'price': pd.np.mean,
        'cost_price': pd.np.mean,
        'quantity': pd.np.sum
    }
)

# calculate average profit margins
category_profit_margin_df['avg_profit_margin'] = category_profit_margin_df.apply(lambda row: round((row['price'] - row['cost_price']) / row['price'] * 100, 2), axis=1)

# filter for only the necessary columns
category_profit_margin_df = category_profit_margin_df[['product_category', 'avg_profit_margin', 'quantity']]

# sort data by avg_profit_margin descending and product_name ascending
category_profit_margin_df = category_profit_margin_df.sort_values('avg_profit_margin', ascending=False)

# reset index for cosmetic reasons
category_profit_margin_df = category_profit_margin_df.reset_index(drop=True)

# filter only the top 20 results
category_profit_margin_df = category_profit_margin_df.head(20)

# write result as csv
write_path = os.path.join(WRITE_DIR, 'category_profit_margin_1.csv')
category_profit_margin_df.to_csv(write_path, index=False)

print(category_profit_margin_df)

                      product_category  avg_profit_margin  quantity
0                               Google              60.02       175
1                              Bottles              58.76       613
2                           Gift Cards              57.98        77
3                                  Fun              57.74       433
4                          Nest-Canada              54.99       135
5                             Headgear              54.65       940
6                               Office              52.65     29105
7                             Nest-USA              52.63      5258
8                            More Bags              52.02        37
9                                 Waze              51.95       383
10                           Lifestyle              51.73      6884
11                             Apparel              51.48      9416
12                           Backpacks              50.98        44
13  ${productitem.product.origCatName}          

<ipython-input-34-8a00ff652665>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  'price': pd.np.mean,
<ipython-input-34-8a00ff652665>:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  'cost_price': pd.np.mean,
<ipython-input-34-8a00ff652665>:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  'quantity': pd.np.sum


## Get Top 20 SKUs with the highest profit margin
### Validate data
- Confirm the assumption that each SKU has only one product name
- If not, when grouping, data would be incorrect

In [35]:
# get df of unique combinations of sku and product_name
check_df = transactions_df[['sku', 'product_name']].drop_duplicates()

# count the number of product_name associated with each sku
check_df = check_df.groupby('sku', as_index=False).agg({'product_name': 'count'})

# halt programme if any discrepancies are found
assert check_df[check_df['product_name'] > 1].empty, 'SKUs with same id but different product names found'

In [46]:
# group data by sku as well as product_name
sku_profit_margin_df = transactions_df.groupby(['sku', 'product_name'], as_index=False).agg(
    {
        'price': pd.np.mean,
        'cost_price': pd.np.mean,
        'quantity': pd.np.sum
    }
)

# calculate average profit margins
sku_profit_margin_df['avg_profit_margin'] = sku_profit_margin_df.apply(lambda row: round((row['price'] - row['cost_price']) / row['price'] * 100, 2), axis=1)

# filter for only the necessary columns
sku_profit_margin_df = sku_profit_margin_df[['sku', 'product_name', 'avg_profit_margin', 'quantity']]

# sort data by avg_profit_margin descending and product_name ascending
sku_profit_margin_df = sku_profit_margin_df.sort_values(['avg_profit_margin', 'product_name'], ascending=[False, True])

# set minimum requirement for quantity
sku_profit_margin_df = sku_profit_margin_df[sku_profit_margin_df['quantity'] >= 500]

# reset index for cosmetic reasons
sku_profit_margin_df = sku_profit_margin_df.reset_index(drop=True)

# filter only the top 20 results
sku_profit_margin_df = sku_profit_margin_df.head(20)

# write result as csv
write_path = os.path.join(WRITE_DIR, 'sku_profit_margin.csv')
sku_profit_margin_df.to_csv(write_path, index=False)

print(sku_profit_margin_df)

               sku                                       product_name  avg_profit_margin  quantity
0   GGOEGFSR022099                                   Google Kick Ball              60.21      1519
1   GGOEGHGH019699                                  Google Sunglasses              60.09      1480
2   GGOEGDHC074099           Google 17oz Stainless Steel Sport Bottle              60.01       660
3   GGOEGHGR019499                                  Google Sunglasses              60.01       670
4   GGOEGOLC014299                       Google Metallic Notebook Set              60.00      1295
5   GGOENEBJ079499  Nestå¨ Learning Thermostat 3rd Gen-USA - Stain...              59.99      1003
6   GGOEGBJC019999                           Collapsible Shopping Bag              59.97       988
7   GGOEGFKA022299                               Keyboard DOT Sticker              59.67       805
8   GGOEGOCC017599                         Recycled Paper Journal Set              57.72       973
9   GGOEGF